In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

4431872/4422102 [==============================] - 0s 0us/step


## $\ell_1$과 $\ell_2$ 규제

In [3]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 16s 9ms/step - loss: 1.5677 - accuracy: 0.8121 - val_loss: 0.7044 - val_accuracy: 0.8364
Epoch 2/2
1719/1719 [==============================] - 14s 8ms/step - loss: 0.7193 - accuracy: 0.8269 - val_loss: 0.6985 - val_accuracy: 0.8390


In [5]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 14s 7ms/step - loss: 1.6722 - accuracy: 0.8124 - val_loss: 0.7165 - val_accuracy: 0.8294
Epoch 2/2
1719/1719 [==============================] - 12s 7ms/step - loss: 0.7221 - accuracy: 0.8263 - val_loss: 0.6812 - val_accuracy: 0.8426


### 드롭아웃

In [27]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 13s 7ms/step - loss: 0.5768 - accuracy: 0.8033 - val_loss: 0.3694 - val_accuracy: 0.8670
Epoch 2/2
1719/1719 [==============================] - 11s 7ms/step - loss: 0.4181 - accuracy: 0.8458 - val_loss: 0.3357 - val_accuracy: 0.8704


### 알파 드롭아웃 - SELU에서는 드롭아웃이 네트워크의 자기 정규화를 망칠 수 있어, 알파 드롭아웃을 사용해야 함.

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.6662 - accuracy: 0.7590 - val_loss: 0.5307 - val_accuracy: 0.8478
Epoch 2/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5560 - accuracy: 0.7955 - val_loss: 0.5249 - val_accuracy: 0.8538
Epoch 3/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5246 - accuracy: 0.8070 - val_loss: 0.5664 - val_accuracy: 0.8470
Epoch 4/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5029 - accuracy: 0.8129 - val_loss: 0.4537 - val_accuracy: 0.8640
Epoch 5/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4927 - accuracy: 0.8183 - val_loss: 0.4718 - val_accuracy: 0.8612
Epoch 6/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4794 - accuracy: 0.8233 - val_loss: 0.4679 - val_accuracy: 0.8606
Epoch 7/20
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4685 - accuracy: 0.8253 - val_loss: 0.4438 - val_accuracy:

In [8]:
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.5220 - accuracy: 0.8573


[0.5219749808311462, 0.8572999835014343]

In [9]:
model.evaluate(X_train_scaled, y_train)

1719/1719 [==============================] - 4s 2ms/step - loss: 0.3981 - accuracy: 0.8776


[0.3980788588523865, 0.8776181936264038]

In [10]:
history = model.fit(X_train_scaled, y_train)

1719/1719 [==============================] - 7s 4ms/step - loss: 0.4184 - accuracy: 0.8439


### 몬테카를로 드롭아웃

In [11]:
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

In [12]:
np.round(model.predict(X_test_scaled[:1]), 2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [13]:
np.round(y_probas[:, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.9 , 0.  , 0.02, 0.  , 0.08]],

       [[0.  , 0.  , 0.01, 0.  , 0.01, 0.16, 0.01, 0.2 , 0.  , 0.62]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.  , 0.94]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.98]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.27, 0.  , 0.16, 0.  , 0.57]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.02, 0.  , 0.96]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.08, 0.  , 0.08, 0.  , 0.84]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.69, 0.  , 0.29]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.01, 0.  , 0.98]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.53, 0.  , 0.37, 0.  , 0.1 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.02, 0.  , 0.68]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.02, 0.  , 0

In [16]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.15, 0.01, 0.19, 0.  , 0.25]],
      dtype=float32)

In [17]:
y_pred = np.argmax(y_proba, axis=1)

In [18]:
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.8685

In [19]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [34]:
mc_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MCAlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    MCAlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    MCAlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = mc_model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.6617 - accuracy: 0.7592 - val_loss: 0.5748 - val_accuracy: 0.7960
Epoch 2/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5546 - accuracy: 0.7974 - val_loss: 0.5578 - val_accuracy: 0.7884
Epoch 3/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5251 - accuracy: 0.8051 - val_loss: 0.5065 - val_accuracy: 0.8134
Epoch 4/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5050 - accuracy: 0.8123 - val_loss: 0.4866 - val_accuracy: 0.8246
Epoch 5/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4910 - accuracy: 0.8181 - val_loss: 0.4908 - val_accuracy: 0.8326
Epoch 6/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4829 - accuracy: 0.8218 - val_loss: 0.4655 - val_accuracy: 0.8288
Epoch 7/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4738 - accuracy: 0.8234 - val_loss: 0.4771 - val_accuracy:

In [35]:
mc_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 mc_alpha_dropout (MCAlphaDr  (None, 784)              0         
 opout)                                                          
                                                                 
 dense_20 (Dense)            (None, 300)               235500    
                                                                 
 mc_alpha_dropout_1 (MCAlpha  (None, 300)              0         
 Dropout)                                                        
                                                                 
 dense_21 (Dense)            (None, 100)               30100     
                                                                 
 mc_alpha_dropout_2 (MCAlpha  (None, 100)            

In [22]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [23]:
mc_model.set_weights(model.get_weights())

In [24]:
np.round(np.mean([mc_model.predict(X_test_scaled[:1]) for sample in range(100)], axis=0), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.12, 0.  , 0.44, 0.  , 0.44]],
      dtype=float32)

### 맥스 노름 - 가중치가 일정 범위 이상으로 높아지는 것을 방지

In [14]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))

In [15]:
MaxNormDense = partial(keras.layers.Dense,
                       activation="selu", kernel_initializer="lecun_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MaxNormDense(300),
    MaxNormDense(100),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 14s 7ms/step - loss: 0.4734 - accuracy: 0.8337 - val_loss: 0.3756 - val_accuracy: 0.8654
Epoch 2/2
1719/1719 [==============================] - 13s 7ms/step - loss: 0.3579 - accuracy: 0.8689 - val_loss: 0.3890 - val_accuracy: 0.8584
